In [1]:
import torch
from torch.nn.functional import elu,relu,leaky_relu
import braindecode
from braindecode.models import *
from braindecode.models.functions import squeeze_final_output,square,safe_log
from skorch.dataset import Dataset
from skorch.callbacks import Checkpoint
from skorch.helper import predefined_split
from config import *
from sklearn.metrics import roc_auc_score
from mne import set_log_level
set_log_level(False)
device = 'cuda' if cuda else 'cpu'

Tensorflow not install, you could not use those pipelines


In [2]:
#Loads preprocessed data from mat files
import scipy
import numpy as np
inputs=scipy.io.loadmat("E:/train_set_1.mat")
X=inputs["x"]
y=inputs["y"].squeeze()

inputs=scipy.io.loadmat("E:/train_set_2.mat")
X=np.concatenate((X,inputs["x"]),axis=0)
y=np.concatenate((y,inputs["y"].squeeze()),axis=0)
input_time_length=X.shape[-1]
inputs=scipy.io.loadmat("E:/test_set.mat")
test_x=inputs["x"]
test_y=inputs["y"].squeeze()
del inputs

In [121]:
#We will now train the model by taking pairs or combinations of channels and passing their entire length.
ch_names=['A1', 'A2', 'C3', 'C4', 'CZ', 'F3', 'F4', 'F7', 'F8', 'FP1','FP2', 'FZ', 'O1', 'O2','P3', 'P4', 'PZ', 'T3', 'T4', 'T5', 'T6']
picked_ch=[5]
input_time_length=X.shape[-1]
train_x=X[:,picked_ch]
n_chans=len(picked_ch)
eval_x=test_x[:,picked_ch]
print(f'Channels chosen:{[ch_names[ch] for ch in picked_ch]}')

Channels chosen:['F3']


In [122]:
#We will convert the 1d array into a 2d matrix by converting from (1,60000) to (10,6000) and then passing them through the network
#Due to conv_time_spat layer in model, it would be better to have it so that first 10 entries are in channel 1, next 10 in channel 2 etc
#So that channel 1 will have 10,110,210 entries and so on
n_chans=10
input_time_length=input_time_length//n_chans
train_x=train_x.reshape(len(train_x),n_chans,input_time_length,order='F')
eval_x=eval_x.reshape(len(eval_x),n_chans,input_time_length,order='F')

In [5]:
model_name='shallow_deep'

In [40]:
n_classes = 2
criterion=torch.nn.NLLLoss
if model_name=="shallow":
    optimizer_lr = 0.0000625
    optimizer_weight_decay = 0
    pool_time_length=150
    pool_time_stride=50
    #The final conv length is auto to ensure that output will give two values for single EEG window
    model = ShallowFBCSPNet(n_chans,
                                    n_classes,
                                    n_filters_time=n_start_chans,
                                    n_filters_spat=n_start_chans,
                                    input_window_samples=input_time_length,
                                    pool_time_length=pool_time_length,
                                    pool_time_stride=pool_time_stride,
                                    final_conv_length='auto',)
    test=torch.ones(size=(7,n_chans,input_time_length))
    out=model.forward(test)
    print(out.shape)
    del test,out
elif model_name=="deep":
    optimizer_lr = init_lr
    optimizer_weight_decay = 0
    model = Deep4Net(n_chans, n_classes,
                         n_filters_time=n_start_chans,
                         n_filters_spat=n_start_chans,
                         input_window_samples=input_time_length,
                         n_filters_2 = int(n_start_chans * n_chan_factor),
                         n_filters_3 = int(n_start_chans * (n_chan_factor ** 2.0)),
                         n_filters_4 = int(n_start_chans * (n_chan_factor ** 3.0)),
                         final_conv_length='auto',
                        stride_before_pool=True)
    test=torch.ones(size=(7,n_chans,input_time_length))
    out=model.forward(test)
    print(out.shape)
    del test,out
elif model_name=="deep_smac" or model_name == 'deep_smac_bnorm':
    optimizer_lr = 0.0000625
    if model_name == 'deep_smac':
            do_batch_norm = False
    else:
        do_batch_norm = True
    drop_prob = 0.244445
    filter_length_2 = 12
    filter_length_3 = 24
    filter_length_4 = 36
    filter_time_length = 21
    #final_conv_length = 1
    first_nonlin = elu
    first_pool_mode = 'mean'
    later_nonlin = elu
    later_pool_mode = 'mean'
    n_filters_factor = 1.679066
    n_filters_start = 32
    pool_time_length = 3
    pool_time_stride = 3
    split_first_layer = True
    n_chan_factor = n_filters_factor
    n_start_chans = n_filters_start
    model = Deep4Net(n_chans, n_classes,
            n_filters_time=n_start_chans,
            n_filters_spat=n_start_chans,
            input_window_samples=input_time_length,
            n_filters_2=int(n_start_chans * n_chan_factor),
            n_filters_3=int(n_start_chans * (n_chan_factor ** 2.0)),
            n_filters_4=int(n_start_chans * (n_chan_factor ** 3.0)),
            final_conv_length='auto',
            batch_norm=do_batch_norm,
            drop_prob=drop_prob,
            filter_length_2=filter_length_2,
            filter_length_3=filter_length_3,
            filter_length_4=filter_length_4,
            filter_time_length=filter_time_length,
            first_conv_nonlin=first_nonlin,
            first_pool_mode=first_pool_mode,
            later_conv_nonlin=later_nonlin,
            later_pool_mode=later_pool_mode,
            #pool_time_length=pool_time_length,
            #pool_time_stride=pool_time_stride,
            split_first_layer=split_first_layer,
            stride_before_pool=True)
elif model_name=="shallow_deep":
    drop_prob = 0.244445
    filter_length_2 = 12
    filter_length_3 = 14
    filter_length_4 = 32
    n_filters_factor = 1.679066
    n_filters_start = 32
    split_first_layer = True
    n_chan_factor = n_filters_factor
    n_start_chans = n_filters_start

    optimizer_lr = 0.0000625
    optimizer_weight_decay = 0
    conv_time_length=25
    first_conv_nonlin=relu
    first_pool_nonlin=safe_log
    later_conv_nonlin=elu
    later_pool_nonlin=safe_log
    first_pool_mode = 'mean'
    later_pool_mode = 'mean'
    pool_time_length=15
    model = Deep4Net(n_chans, n_classes,
                            input_time_length,
                            n_filters_time=n_start_chans,
                            n_filters_spat=n_start_chans,
                            n_filters_2 = int(n_start_chans * n_chan_factor),
                            n_filters_3 = int(n_start_chans * (n_chan_factor ** 2.0)),
                            n_filters_4 = int(n_start_chans * (n_chan_factor ** 3.0)),
                            final_conv_length='auto',
                            first_pool_nonlin=first_pool_nonlin,
                            first_conv_nonlin=first_conv_nonlin,
                            #later_pool_nonlin=later_pool_nonlin,
                            #later_conv_nonlin=later_conv_nonlin,
                            filter_time_length=conv_time_length,
                            pool_time_length=pool_time_length,
                            first_pool_mode=first_pool_mode,
                            later_pool_mode=later_pool_mode,
                            split_first_layer=split_first_layer,
                            drop_prob=drop_prob,
                            filter_length_2=filter_length_2,
                            filter_length_3=filter_length_3,
                            filter_length_4=filter_length_4,
                            )
elif model_name=="attention":
    optimizer_lr = 0.0000625
    optimizer_weight_decay = 0
    model=ATCNet(n_chans,n_classes,input_time_length//sampling_freq,sampling_freq,concat=True,tcn_depth=4)
    test=torch.ones(size=(7,n_chans,input_time_length))
    out=model.forward(test)
    print(out.shape)
    del test,out
if cuda:
    model.cuda()

print(model_name)
print(model)

shallow_deep
Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
Deep4Net (Deep4Net)                      [1, 10, 6000]             [1, 2]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 10, 6000]             [1, 10, 6000, 1]          --                        --
├─Rearrange (dimshuffle): 1-2            [1, 10, 6000, 1]          [1, 1, 6000, 10]          --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 6000, 10]          [1, 32, 5976, 1]          11,072                    --
├─BatchNorm2d (bnorm): 1-4               [1, 32, 5976, 1]          [1, 32, 5976, 1]          64                        --
├─Expression (conv_nonlin): 1-5          [1, 32, 5976, 1]          [1, 32, 5976, 1]          --                        --
├─AvgPool2dWithConv (pool): 1-6          [1, 32, 5976, 1]          [1, 32, 1988, 1]          --                        [15,

c:\Users\lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\braindecode\models\base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [127]:
monitor = lambda net: any(net.history[-1, ('valid_accuracy_best','valid_f1_best','valid_loss_best')])
cp=Checkpoint(monitor='valid_f1_best',dirname='model',f_params=f'chanbest_param.pkl',
               f_optimizer=f'chanbest_opt.pkl', f_history=f'chanbest_history.json')
classifier = braindecode.EEGClassifier(
    model,
    criterion=criterion,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(Dataset(eval_x,test_y)),
    optimizer__lr=optimizer_lr,
    #optimizer__weight_decay=optimizer_weight_decay,
    iterator_train__shuffle=True,
    batch_size=batch_size,
    device=device,
    callbacks=["accuracy","f1",'roc_auc',cp],#Try ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=20))
    warm_start=True,
    )
classifier.initialize()

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  Deep4Net (Deep4Net)                      [1, 10, 6000]             [1, 2]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 10, 6000]             [1, 10, 6000, 1]          --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 10, 6000, 1]          [1, 1, 6000, 10]          --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 6000, 10]          [1, 32, 5976, 1]          11,072                    --
  ├─BatchNorm2d (bnorm): 1-4               [1, 32, 5976, 1]          [1, 32, 5976, 1]          64                        --
  ├─Expression (conv_nonlin): 1-5 

In [ ]:
classifier.load_params(f_params=f'model/chanbest_param.pkl',f_optimizer=f'model/chanbest_opt.pkl', f_history=f'model/chanbest_history.json')
print("Paramters Loaded")

In [87]:
test=np.random.rand(7,n_chans,input_time_length)
out=classifier.predict(test)
print(out)

[0 0 0 0 0 0 0]


In [ ]:
#   Accuracy    F1-score       Loss              AUC
#A1:0.7481      0.7344        0.5478           0.8070
#A2:0.7556      0.7130        0.6907           0.7879
#C3:0.7259      0.7040        0.6569           0.7496
#C4:0.7111      0.6286        0.8119           0.7738
#CZ:0.6963      0.6306        0.7615           0.7837
#F3:0.7111      0.6549        0.8018           0.7540
#F4:
#F7:
#F8:
#FP1:
#FP2:
#FZ:
#O1:
#O2:
#P3:
#P4:
#PZ:
#T3:
#T4:
#T5:
#T6:

In [128]:
classifier.fit(train_x,y,epochs=20)

  epoch    train_accuracy    train_f1    train_loss    train_roc_auc    valid_acc    valid_accuracy    valid_f1    valid_loss    valid_roc_auc    cp     dur
-------  ----------------  ----------  ------------  ---------------  -----------  ----------------  ----------  ------------  ---------------  ----  ------
      1            0.9581      0.8548        0.1254           0.9959       0.6370            0.6370      0.4235        1.3388           0.8118     +  3.2020
      2            0.9760      0.9262        0.1063           0.9949       0.6963            0.6963      0.5859        0.9846           0.7905     +  3.0500
      3            0.9844      0.9536        0.1072           0.9984       0.6667            0.6667      0.5263        0.9413           0.7879        3.0910
      4            0.9838      0.9486        0.0912           0.9990       0.6889            0.6889      0.5435        1.0371           0.8352        2.9800
      5            0.9689      0.9113        0.0950       

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  Deep4Net (Deep4Net)                      [1, 10, 6000]             [1, 2]                    --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 10, 6000]             [1, 10, 6000, 1]          --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 10, 6000, 1]          [1, 1, 6000, 10]          --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 6000, 10]          [1, 32, 5976, 1]          11,072                    --
  ├─BatchNorm2d (bnorm): 1-4               [1, 32, 5976, 1]          [1, 32, 5976, 1]          64                        --
  ├─Expression (conv_nonlin): 1-5 

In [ ]:
classifier.save_params(f_params=f'model/chanbest_param.pkl',f_optimizer=f'model/chanbest_opt.pkl', f_history=f'model/chanbest_history.json')

In [ ]:
#This block finds the accuracy, f1 score and roc auc of the valid/test set
pred_labels=classifier.predict(eval_x)
auc=roc_auc_score(test_y,classifier.predict_proba(eval_x)[:,1])
accuracy=np.mean(pred_labels==test_y)
tp=np.sum(pred_labels*test_y)
precision=tp/np.sum(pred_labels)
recall=tp/np.sum(test_y)
f1=2*precision*recall/(precision+recall)

print(model_name)
print(f"Accuracy:{accuracy}")
print(f"F1-Score:{f1}")
print(f"roc_auc score:{auc}")